<h1> Lecture 23 

Data Science 8, Spring 2021 </h1>

<h3>
<b>
<ul>
<li>Percentiles  </li><br>
    
<li>Estimation  </li><br>

<li>Bootstrap  </li><br>
</ul>
</b>
</h3>

In [ ]:
from datascience import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
plots.rcParams["patch.force_edgecolor"] = True

#The following allows porting images into a Markdown window
#Syntax: ![title](image_name.png)
from IPython.display import Image

### Percentiles

<h3>Determine manually the 55th percentile of the set<br>
    
$$x=[43, 20, 51, 7, 28, 34].$$
</h3>

<h4>Step 0: Create the data array:</h4>

In [ ]:
x = make_array(43, 20, 51, 7, 28, 34)
x

<h5>Size of the data set</h5>

In [ ]:
data_set_size=len(x)
data_set_size

<h4>Step 1: Sort the data.</h4>

In [ ]:
# Step 1. Sort the data
x_sorted=np.sort(x)
x_sorted

<h4>Step 2: Set the desired percentile $p$</h4>

In [ ]:
p=55 #pth percentile

<h4>Step 3: Determine where the $p^\textsf{th}$ percentile is.</h4>

In [ ]:
(p/100)*data_set_size

In [ ]:
pth_percentile_count = np.ceil((p/100)*data_set_size)
pth_percentile_count

<h4>Step 4: Grab the item from the array</h4>

<h4>If we're not careful, we get an error message:</h4>

In [ ]:
#Recall that Python indexing of array elements starts at 0.
#Hence, the subtraction of 1
x_sorted.item(pth_percentile_count-1)

In [ ]:
pth_percentile_count=int(np.ceil((p/100)*data_set_size))
pth_percentile_count

<h5>Now we should get the correct answer:</h5>

In [ ]:
#Recall that Python indexing of array elements starts at 0.
#Hence, the subtraction of 1
x_sorted.item(pth_percentile_count-1)

<h3>Alternative Method Using a Single Line of Code: <tt>percentile</tt> Function</h3>
<h4>Determine where the $p^\textsf{th}$ percentile is.</h4>

In [ ]:
percentile(p, x)

<h4>Notice in the line above we do NOT need to sort the array.  The <tt>percentile</tt> function does it for us.</h5>

<h3>Determine manually the 50th percentile (i.e., median) of the set<br>
    
$$x=[43, 20, 51, 7, 28, 34].$$
</h3>

In [ ]:
x_sorted

<h3>Caution! NumPy Median: The function <tt>np.median</tt> behaves differently for even-length arrays.</h3>

In [ ]:
percentile(50, x)  #Recall: We need NOT sort the data separately

In [ ]:
np.median(x) #We need NOT sort the data separately

<h4>NumPy averages the middle two numbers in an even-length array:<br><br>
    
$$\textsf{np.median}(x)=\displaystyle \frac{28+34}{2}=31.$$

Conforms to the interpretation that 50% of the values are above, and 50% are below, the median.<br>

Also, even if all the elements of an array are integers, <tt>np.median()</tt> returns a <tt>float</tt>.
</h4>


Going forward, to compute the median, instead of <tt>np.median(x)</tt> use <tt>percentile(50,x)</tt>.  The latter is consistent with the [description](https://www.inferentialthinking.com/chapters/13/1/Percentiles.html) given in the lecture notes for the course. 




SLIDE: Discussion Question

<h3>Discussion Question</h3>

In [ ]:
s = make_array(1, 7, 3, 9, 5)
s

In [ ]:
s_sorted = np.sort(s)
s_sorted

In [ ]:
percentile(10,s)

In [ ]:
percentile(10,s)==0

<h4>Note: The number 0 is not in the data set.  Any percentile of a data set is always a member of that set.</h4>

In [ ]:
percentile(40, s)

In [ ]:
(39/100)*5

In [ ]:
np.ceil((39/100)*5)

In [ ]:
int(np.ceil((39/100)*5))

In [ ]:
s_sorted[int(np.ceil((39/100)*5))-1]

<h4>All of that in one line of code</h4>

In [ ]:
percentile(39, s)

In [ ]:
percentile(39, s) == percentile(40, s)

In [ ]:
percentile(41, s)

In [ ]:
percentile(40, s) == percentile(41, s)

In [ ]:
percentile(50, s)

In [ ]:
percentile(50, s) == 5

<h4>We expect the same answer (but in <tt>float</tt>) from <tt>np.median()</tt> for this odd-length array.</tt></h4>

In [ ]:
np.median(s)

SLIDE: Estimation

<h2>City of San Francisco Employees: Sample Median</h2>

In [ ]:
sf = Table.read_table('san_francisco_2015.csv')
sf

<h3>Declutter: Remove extraneous columns</h3>

In [ ]:
sf=sf.select('Job Family','Job','Total Compensation')
sf

<h3>Who had the top five 'Total Compensation' and what were their jobs?</h3>

In [ ]:
# Who is making the most money
sf.sort('Total Compensation', descending=True).show(5)

<h3>Who had the lowest five 'Total Compensation' and what were their jobs?</h3>

In [ ]:
# Who is making the least money
sf.sort('Total Compensation', descending=False).show(5)

<h3>Clearly, the data must be cleaned, so spurious rows like the above are removed.</h3>

<h3>Let's impose a lower cutoff (minimum salary cutoff) for salaries.<br>
    
Assume half-time employment&mdash;20 hours a week.<br>
    
Assume minimum wage of $10/hour.<br>
</h3>

![title](2015-min-wage-schedule-san-francisco.png)

[Source: GovDocs](https://www.govdocs.com/san-francisco-2015-minimum-wage-increase/)

<h3>Assume a round figure of $10/hour for minimum wage in 2015.</h3>

In [ ]:
min_wage_per_hour = 10  #minimum hourly wage
hours_per_week    = 20  #half-time employee
weeks_per_year    = 52

min_salary=min_wage_per_hour * hours_per_week * weeks_per_year
min_salary

<h3>Keep only the rows where 'Total Compensation' is at least <tt>min_salary</tt>:</h3>

In [ ]:
sf = sf.where('Total Compensation', are.above(min_salary))#.sort('Total Compensation', descending=True)
sf

<h3>Population Histogram</h3>

<h4>Median 'Total Compensation' of the entire data set:</h4>

In [ ]:
pop_median = percentile(50, sf.column('Total Compensation'))
pop_median 

<h4>Maximum 'Total Compensation' of the entire data set:</h4>

In [ ]:
pop_max = max(sf.column('Total Compensation'))
pop_max

<h4>Minimum 'Total Compensation' of the entire data set:</h4>

In [ ]:
pop_min = sf.column('Total Compensation').min()
pop_min

<h4>Set the histogram bins.</h4>

In [ ]:
sf_bins = np.arange(0, 700000, 25000)
sf_bins

<h4>Plot the Population Distribution</h4>

In [ ]:
sf.hist('Total Compensation', bins=sf_bins)
plots.title('Population Distribution');

<h3>Now take a sample of size <tt>sample_size</tt> <i>different</i> employees:</h3>

In [ ]:
sample_size=300
sample_size

In [ ]:
our_sample = sf.sample(sample_size, with_replacement=False)
our_sample.show(5)

<h4>Median 'Total Compensation' for our sample is:</h4>

In [ ]:
sample_median = percentile(50, our_sample.column('Total Compensation'))
sample_median

<h4>Compare with the population median.</h4>

In [ ]:
pop_median

SLIDE: Variability of the Estimate

<h3>Sample Histogram</h3>

<h4>Maximum 'Total Compensation' in Our Sample:</h4>

In [ ]:
sample_max = our_sample.column('Total Compensation').max()
sample_max

<h4>Minimum 'Total Compensation' in Our Sample:</h4>

In [ ]:
sample_min = our_sample.column('Total Compensation').min()
sample_min

<h4>Plot the Sample Distribution (using the same bins as for the population)</h4>

In [ ]:
our_sample.hist('Total Compensation', bins=sf_bins)
plots.title('Sample Distribution');

# Variability of the Estimate

In [ ]:
def generate_sample_median(sample_size):
    our_sample = sf.sample(sample_size, with_replacement=False)
    return percentile(50, our_sample.column('Total Compensation'))

In [ ]:
sample_size=300
sample_median = generate_sample_median(sample_size)
sample_median

In [ ]:
error = sample_median - pop_median
error

<h4>Repeat the cell below manually a few times.</h4>

In [ ]:
sample_median = generate_sample_median(sample_size)
error = sample_median - pop_median
error

# Quantifying Uncertainty

In [ ]:
num_simulations = 1000
sample_size = 300

#Create a blank sample_medians array
sample_medians = make_array()

for i in np.arange(num_simulations):
    new_median = generate_sample_median(sample_size)
    sample_medians = np.append(sample_medians, new_median)

<h4>Minimum Sample Median</h4>

In [ ]:
min(sample_medians)

<h4>Maximum Sample Median</h4>

In [ ]:
max(sample_medians)

In [ ]:
median_bins = np.arange(90000, 125001, 2500)
Table().with_column(
    'Sample Medians', sample_medians).hist(bins = median_bins)
#Superimpose the true population median
plots.scatter(pop_median, -1e-6, color="red");

<h4>Create an array of sample errors:</h4>

In [ ]:
sample_errors = sample_medians - pop_median

<h4>Minimum Sample Error:</h4>

In [ ]:
min(sample_errors)

<h4>Maximum Sample Error:</h4>

In [ ]:
max(sample_errors)

In [ ]:
bin_width=2500

error_bins = np.arange(-15000, 12501, bin_width)

Table().with_column(
    'Errors', sample_errors).hist(bins = error_bins)

plots.scatter(0, -1e-6, color="red");

SLIDE: Where to Get Another Sample?

# Bootstrap

In [ ]:
# Take a bootstrap (re)sample of size 300, WITH replacement
sample_size = 300


boot_sample = our_sample.sample(sample_size, with_replacement=True)
boot_sample.hist('Total Compensation', bins=sf_bins)
plots.title('Bootstrap sample');

print("Population Median =       ", pop_median)
print("Our Sample Median =       ", sample_median)
print("Bootstrap Sample Median = ", 
      percentile(50,boot_sample.column('Total Compensation')))

In [ ]:
def one_bootstrap_median():
    boot_resample = our_sample.sample()
    return percentile(50, boot_resample.column('Total Compensation'))

In [ ]:
num_simulations = 1000

bootstrap_medians = make_array()
for i in np.arange(num_simulations):
    new_median = one_bootstrap_median()
    bootstrap_medians = np.append(bootstrap_medians, new_median)

In [ ]:
Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=median_bins)

plots.scatter(pop_median, -1e-6, color="red");
plots.scatter(sample_median, -1e-6, color="blue");

## Confidence Intervals

The confidence interval is an interval based on the middle 95% of bootstrap samples.  The interval will be shown in yellow, the sample median (our estimate) in blue, and the true population median (the parameter) in red.

In [ ]:
left = percentile(2.5, bootstrap_medians)
right = percentile(97.5, bootstrap_medians)

Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=median_bins)

plots.plot([left, right], [-1e-6,-1e-6], color="gold",lw=3, zorder=1);
plots.scatter(pop_median, -1e-6, color="red", zorder=2);
plots.scatter(sample_median, -1e-6, color="blue", zorder=2);